In [21]:
from scipy import io
import numpy as np
# from matplotlib.mlab import PCA
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import normalize



from sklearn.decomposition import PCA
# from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt




data = io.loadmat("oc_514.mat")

dataset = data['x'][0][0][0]
#finding the optimal bandwidth using cross validation
def apply_KDE(train_dataset, test_dataset=None):
    if test_dataset is None:
        test_dataset = train_dataset
    params = {'bandwidth' : np.logspace(-1,1, 20)}
    grid = GridSearchCV(KernelDensity(), params)
    grid.fit(train_dataset)

    print "optimal bandwidth in original data", grid.best_estimator_.bandwidth
    kde_skl = KernelDensity( bandwidth=grid.best_estimator_.bandwidth)
    kde_skl.fit(train_dataset)
    log_pdf = kde_skl.score_samples(test_dataset)

    return log_pdf
    
log_pdf = apply_KDE(dataset)
print log_pdf
print np.exp(log_pdf)[0:20]

optimal bandwidth in original data 7.84759970351
[-834.24291165 -834.24291165 -834.24291165 -834.24291165 -834.24291165
 -834.24291165 -834.24291165 -834.24291165 -834.24291165 -834.24291165
 -834.24291165 -834.24291165 -834.24291165 -834.24291165 -834.24291165
 -834.24291165 -834.24291165 -834.24291165 -834.24291165 -834.24291165
 -834.24291165 -834.24291165 -834.24291165 -834.24291165 -834.24291165
 -834.24291165 -834.24291165 -834.24291165 -834.24291165 -834.24291165
 -834.24291165 -834.24291165 -834.24291165 -834.24291165 -834.24291165
 -834.24291165 -834.24291165 -834.24291165 -834.24291165 -834.24291165
 -834.24291165 -834.24291165 -834.24291165 -834.24291165 -834.24291165
 -834.24291165 -834.24291165 -834.24291165 -834.24291165 -834.24291165
 -834.24291165 -834.24291165 -834.24291165 -834.24291165 -834.24291165
 -834.24291165 -834.24291165 -834.24291165 -834.24291165 -834.24291165
 -834.24291165 -834.24291165 -834.24291165 -834.24291165 -834.24291165
 -834.24291165 -834.24291165

The output is in log format and if we calculate the probability density then its coming out to be 0. KDE does not perform well with higher number of feature and from the data source it has been learnt that 50 features are sufficient to find the covariance in the dataset upto 98-99 percent. Thus stripping down to 50 features can be a good idea.

Since different feature are in different format so normalization of all the features would be helpful before doing PCA.

In [22]:
normalised_dataset = normalize(dataset)
#applying PCA for 50 features
pca = PCA(n_components=50)
dataset_pca= pca.fit_transform(normalised_dataset)
log_pdf_pca = apply_KDE(dataset_pca)
print np.exp(log_pdf_pca)

optimal bandwidth in original data 0.1
[  4.52586998e+27   1.23652816e+28   2.92889951e+27   6.73733831e+27
   2.67527331e+27   1.37630699e+28   3.00244926e+27   1.74174712e+28
   1.64329188e+28   3.91180145e+27   1.57606704e+28   4.62981599e+27
   7.91528047e+27   1.53248739e+28   1.77999444e+28   6.77682298e+27
   4.32121835e+27   6.19888266e+27   4.94953080e+27   8.28026911e+27
   9.79982102e+27   3.00520518e+28   1.65659764e+28   1.62184294e+28
   2.77371299e+27   5.91236944e+27   1.03161973e+28   2.68123037e+27
   2.70407941e+27   2.97685044e+27   8.64608650e+27   1.99743510e+28
   1.70504012e+28   1.29957985e+28   1.38311172e+28   1.38747727e+28
   1.00355205e+28   7.85149971e+27   2.99281808e+27   1.41136686e+28
   1.73713401e+28   1.02469068e+28   9.50371168e+27   5.05934377e+27
   4.45683208e+27   9.05843590e+27   1.65114014e+28   1.10006952e+28
   7.59425438e+27   1.26401631e+28   2.76065755e+28   8.32403356e+27
   5.46090286e+27   1.65874940e+28   2.90476977e+27   9.40476571

It provides a better result then previous approach. But it may be over fitting as we are validating the eximation against the training set only.
To really understand the performance, Cross validation can be used. Test set is contructed from the last 80 data points from the whole data set to validate the performance.

In [23]:
# train, test = train_test_split(dataset_pca, test_size = 0.2)
no_of_train_data = 340
train = dataset_pca[0:no_of_train_data]
test = dataset_pca[no_of_train_data:len(dataset_pca)]
log_pdf_pca_cross_validation = apply_KDE(train, test)
print np.exp(log_pdf_pca_cross_validation)

optimal bandwidth in original data 0.1
[  3.98150094e+27   2.86149609e+26   1.93827636e+27   7.54380371e+27
   2.19027900e+27   7.68803266e+27   5.33313925e+26   9.57105516e+27
   2.54904474e+28   1.06267246e+24   5.58173332e+26   6.55003960e+24
   1.42186903e+28   6.42808095e+27   5.84760597e+27   4.26246297e+27
   1.55013106e+26   4.78698866e+27   7.34328652e+25   2.02247765e+25
   1.87871351e+28   4.00858705e+27   1.99755178e+28   1.82700917e+24
   3.35580319e+27   1.72578667e+25   5.00156937e+27   4.82372602e+27
   8.10372943e+26   1.50241029e+28   2.22054802e+28   2.10560421e+27
   4.29356644e+27   2.66226382e+21   1.05174089e+28   8.88546076e+27
   1.32145389e+28   8.41138089e+26   2.93351375e+27   3.64574310e+27
   1.19534753e+27   8.77005540e+27   8.30309757e+27   2.80796279e+27
   6.99272792e+27   3.27211189e+27   4.51852702e+25   1.23857834e+28
   3.25531436e+27   1.05748007e+28   1.96453865e+28   6.10192834e+27
   2.41563502e+23   1.89176676e+28   9.60670173e+27   7.58402670

Since the data set has around 45% of outliers, no function will perform good. Lets see how are the results after removing the outliers.




In [24]:
outlier_info = data['x'][0][0][2]

# train_without_outlier = dataset_pca[0:no_of_train_data]
# test_without_outlier = dataset_pca[no_of_train_data:len(dataset_pca)]
# print dataset_without_outliers
train_dataset_without_outliers = []
for item in range(0,no_of_train_data):
    if(outlier_info[item][0] == 2):
        train_dataset_without_outliers.append(dataset_pca[item])
                                        
print "The size of training dataset without outliers is ",len(train_dataset_without_outliers)
print("Below indices are the outliers in the test data set:")
# indices of the outliers in the test data set
outlier_indices_test = []
for item in range(0,len(dataset_pca)-no_of_train_data):
    if(outlier_info[item+no_of_train_data][0] == 1):
        outlier_indices_test.append(item)

The size of training dataset without outliers is  188
Below indices are the outliers in the test data set:


In [25]:
# since the size of dataset is very small the size of test set is reduced to 25
# train, test = train_test_split(dataset_pca, test_size = 0.1)

log_pdf_pca_cross_validation_without_outlier = apply_KDE(train_dataset_without_outliers, test)
print np.exp(log_pdf_pca_cross_validation_without_outlier)


optimal bandwidth in original data 0.1
[  4.66220683e+27   4.31851339e+25   2.73229258e+27   1.08106909e+28
   3.07717995e+27   9.93956643e+27   7.05116020e+26   1.38202373e+28
   3.36909268e+28   1.38353609e+23   5.66693065e+26   7.68580211e+23
   2.12395066e+28   9.58307916e+27   7.08199427e+27   6.38433648e+27
   1.87545044e+26   6.11450719e+27   1.23994981e+23   1.07072754e+25
   2.46774062e+28   5.70778507e+27   2.84482613e+28   1.56112187e+24
   3.38308793e+27   3.42331763e+23   7.27142306e+27   4.92144701e+27
   1.52653926e+26   1.95092981e+28   3.26152800e+28   2.54653680e+27
   5.99319868e+27   3.99466815e+19   1.48602133e+28   1.27705425e+28
   1.87100865e+28   1.21039939e+27   2.57625113e+27   4.58299687e+27
   1.31835450e+27   1.13720148e+28   9.57293149e+27   3.93669104e+27
   7.51747524e+27   4.11662542e+27   2.12775735e+25   1.65240612e+28
   4.04045564e+27   1.43610312e+28   2.68059342e+28   8.94934741e+27
   2.05249279e+23   2.75446645e+28   1.35058491e+28   4.94588523

Applying the one class SVM. as we have already seen that cross validation is a nice approach to go ahead to check the performance of the algorithm. We will use test and training data set for the one class SVM

# One class SVM


In [27]:
from sklearn import svm 
estimator = svm.OneClassSVM(gamma=0.0001)
estimator.fit(train)
prediction_svm = estimator.predict(test)
print prediction_svm

outlier_indices_test = []
for item in range(0, len(dataset) - no_of_train_data):
    if (outlier_info[item + no_of_train_data][0] == 1):
        outlier_indices_test.append(item)
print(outlier_indices_test)

predicted_outlier_count = 0
correct_outlier_prediction = 0
for item in range(len(prediction_svm)):
    if(item in outlier_indices_test and prediction_svm[item] == -1):
        correct_outlier_prediction += 1
    if prediction_svm[item] == -1:
        predicted_outlier_count += 1

print "Total no of outlier in test dataset = ",len(outlier_indices_test)        
print "Total no of predicted outliers = ",predicted_outlier_count
print "Total no of correct outlier predicted = ", correct_outlier_prediction
print "Total no of inliers predicted as outliers = ", predicted_outlier_count - correct_outlier_prediction
print "Total no of outliers predicted inliers = ",len(outlier_indices_test) - correct_outlier_prediction


[ 1. -1.  1.  1. -1.  1. -1.  1.  1. -1. -1. -1.  1.  1. -1.  1. -1.  1.
 -1. -1.  1. -1.  1. -1. -1. -1.  1. -1. -1.  1.  1. -1.  1. -1.  1. -1.
  1. -1. -1. -1. -1.  1.  1. -1.  1.  1. -1.  1.  1.  1.  1.  1. -1.  1.
  1. -1. -1. -1. -1.  1. -1. -1. -1.  1.  1.  1.  1. -1.  1. -1. -1.  1.
  1.  1.  1. -1. -1. -1. -1.  1.]
[1, 4, 5, 6, 9, 10, 11, 14, 16, 17, 18, 25, 27, 28, 30, 31, 33, 39, 40, 41, 46, 52, 55, 57, 58, 59, 60, 61, 71, 76, 77]
Total no of outlier in test dataset =  31
Total no of predicted outliers =  40
Total no of correct outlier predicted =  25
Total no of inliers predicted as outliers =  15
Total no of outliers predicted inliers =  6


# Local Factor Outlier

In [28]:
from sklearn.neighbors import NearestNeighbors

minPts = 27

# calculating nearest point to minimum points
minPtsNN = NearestNeighbors(n_neighbors=minPts).fit(dataset_pca)
reach_dists, reach_Idx = minPtsNN.kneighbors(dataset_pca)
#print reach_dists

for index in range(minPts):
    reach_dists[:,index] = np.amax(reach_dists,axis=1)
#print reach_dists
#caulating the lrd matrix from the reachability matrix
lrd_minPts = minPts/np.sum(reach_dists, axis=1)
#print lrd_minPts
lof_score = []

#according to the equation of the lof, calculating the lof score
for index in reach_Idx:
    lof_score.append(np.divide(reach_dists[index[1:]], reach_dists[index[0]]).sum()/minPts)

#print lof_score

outlier_threshold = 20
detected_outlier = []
for index in range(len(lof_score)):
    if lof_score[index] > outlier_threshold:
        detected_outlier.append(index)

outlier_indices = []
for item in range(0, len(dataset_pca)):
    if (outlier_info[item][0] == 1):
        outlier_indices.append(item)
#print(outlier_indices)

no_correct_outlier = 0
for index in detected_outlier:
    if index in outlier_indices:
        no_correct_outlier += 1

print "actual no of outliers = ", len(outlier_indices)
print "detected outliers = ", len(detected_outlier)
print "correct outlier detected = ", no_correct_outlier
print "no of outliers detected as inliers = ",len(outlier_indices) - no_correct_outlier
print "no of inliers detected as outliers = ",len(detected_outlier) - no_correct_outlier


actual no of outliers =  183
detected outliers =  338
correct outlier detected =  121
no of outliers detected as inliers =  62
no of inliers detected as outliers =  217
